In [1]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --disable-pip-version-check torch==1.13.1 torchdata==0.5.1 --quiet
!{sys.executable} -m pip install transformers==4.27.2 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.18.0 requires torch

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig

hf_model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(hf_model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
# need to use tokenizer specific to this model
tokenizer = AutoTokenizer.from_pretrained(hf_model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [4]:
input_query = "Hello! I am testing the tokenizer now."

input_tokenized = tokenizer(input_query, return_tensors="pt")

input_untokenized = tokenizer.decode(input_tokenized["input_ids"][0], skip_special_tokens=False)

# Will print the input query after decoded from the token space
print(input_untokenized)

Hello! I am testing the tokenizer now.</s>


In [8]:
# NOTE: the model outputs in tokens, it needs to be decoded through the tokenizer
# Exploratory: prompt with no examples, only direction and a question
input = '''
Context: You are a SQL statement generating machine that takes in natural language
and outputs the equivalent of the request as a SQL query. Only return the SQL query,
do not return any natural language.
There is a database called "store_date", it has two tables called "coffee_sales"
and "pastry_sales" with columns "cost", "item", and "time" in each

Question: How many coffee sales are about $5?
'''
input_tokenized = tokenizer(input, return_tensors="pt")
model_outputs = model.generate(input_tokenized['input_ids'], max_new_tokens=50)
model_output_decoded = tokenizer.decode(model_outputs[0], skip_special_tokens=True)

print(model_output_decoded)

Count the number of coffee sales that are about $5


In [7]:
# As we can see, it is very ineffective without examples